# Run Experiments on Fjord

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

# extract test data
df = pd.read_parquet('datasets/segmented-objects-dataset.parquet')

# Assuming df is your DataFrame with all data
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Local version of get labels
def get_hierarchical_labels(species_index, species_names, genus_names, class_names, binary_names, root):
    if species_index == -1:
        return -1, -1, -1  # Handle cases where species_index is invalid

    species_name = species_names[species_index]
    node = next((n for n in root.descendants if n.name == species_name), None)

    if node is None:
        return -1, -1, -1  # Species not found in the tree

    genus_index, class_index, binary_index = -1, -1, -1
    current_node = node
    while current_node.parent is not None:
        current_node = current_node.parent
        if current_node.rank == 'genus':
            genus_index = genus_names.index(current_node.name) if current_node.name in genus_names else -1
        elif current_node.rank == 'class':
            class_index = class_names.index(current_node.name) if current_node.name in class_names else -1
        elif current_node.rank == 'binary':
            binary_index = binary_names.index(current_node.name) if current_node.name in binary_names else -1

    return genus_index, class_index, binary_index

## Load Model and run experiments

In [2]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from collections import defaultdict
from anytree.importer import JsonImporter
from utils.custom_dataset import CustomDataset

def run_experiment(data_path, classes_file, model_path, ablation, root):
    df = pd.read_parquet(data_path)
    _, test_df = train_test_split(df, test_size=0.3, random_state=42)

    object_names = [line.strip() for line in open(classes_file, 'r')]
    subcategory_names, category_names, binary_names = [], [], []
    for node in root.descendants:
        if node.rank == 'genus':
            subcategory_names.append(node.name)
        elif node.rank == 'class':
            category_names.append(node.name)
        elif node.rank == 'binary':
            binary_names.append(node.name)

    rank_counts = defaultdict(int)
    for node in root.descendants:
        rank_counts[node.rank] += 1
    num_classes_hierarchy = [rank_counts['binary'], rank_counts['class'], rank_counts['genus'], rank_counts['species']]

  # Adjust model import and initialization based on ablation type
    if ablation == 'baseline':
        from models.hierarchical_cnn import HierarchicalCNN
        num_additional_features = 3  # e.g., conf, iou, pred_species
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features)
    elif ablation == 'Yolov8':
        from models.hierarchical_cnn import HierarchicalCNN
        num_additional_features = 3  # e.g., conf, iou, pred_species
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features)
    elif ablation == 'remove_features':
        from models.ablations.remove_features.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy)
    elif ablation == 'attention_removed':
        from models.ablations.attention_removed.hierarchical_cnn import HierarchicalCNN
        num_additional_features = 3  # e.g., conf, iou, pred_species
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features)
    elif ablation == 'decreased_branch_complexity':
        from models.ablations.decreased_branch_complexity.hierarchical_cnn import HierarchicalCNN
        num_additional_features = 3  # e.g., conf, iou, pred_species
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features)
    elif ablation == 'increased_features_complexity':
        from models.ablations.increased_features_complexity.hierarchical_cnn import HierarchicalCNN
        num_additional_features = 3  # e.g., conf, iou, pred_species
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features)
    else:
        raise ValueError(f"Unsupported ablation study: {ablation}")


    model.load_state_dict(torch.load(model_path))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    test_dataset = CustomDataset(test_df, object_names, subcategory_names, category_names, binary_names, root)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    true_labels = {level: [] for level in ['binary', 'class', 'genus', 'species']}
    predictions = {level: [] for level in ['binary', 'class', 'genus', 'species']}

    with torch.no_grad():
        for images, conf, iou, pred_species, species_index, genus_index, class_index, binary_index in test_loader:
            images, conf, iou, pred_species = [x.to(device) for x in [images, conf, iou, pred_species]]

            if ablation == 'Yolov8':
                for idx in pred_species:
                    genus_idx, class_idx, binary_idx = get_hierarchical_labels(idx.item(), object_names, subcategory_names, category_names, binary_names, root)
                    # Storing the hierarchical labels
                    predictions['binary'].append(binary_idx)
                    predictions['class'].append(class_idx)
                    predictions['genus'].append(genus_idx)
                    predictions['species'].append(idx.item())
            else:
                outputs = model(images, conf, iou, pred_species) if ablation != 'remove_features' else model(images)
                for i, output in enumerate(outputs):
                    _, predicted = torch.max(output, 1)
                    level = ['binary', 'class', 'genus', 'species'][i]
                    predictions[level].extend(predicted.cpu().numpy())

            # Store true labels directly from the DataLoader
            true_labels['binary'].extend(binary_index.cpu().numpy())
            true_labels['class'].extend(class_index.cpu().numpy())
            true_labels['genus'].extend(genus_index.cpu().numpy())
            true_labels['species'].extend(species_index.cpu().numpy())

    f1_scores = {level: f1_score(true_labels[level], predictions[level], average='macro') for level in ['binary', 'class', 'genus', 'species']}
    return f1_scores

# Populate Taxonomy
importer = JsonImporter()
with open('datasets/ontology.json', 'r') as f:
    root = importer.read(f)

# Paths and setup
data_path = 'datasets/segmented-objects-dataset.parquet'
classes_file = 'datasets/The Fjord Dataset/fjord/classes.txt'
ablations = ['Yolov8', 'baseline', 'increased_features_complexity', 'remove_features', 'attention_removed', 'decreased_branch_complexity']
alpha_values = [0, 0.2, 0.5, 0.8, 1]

# Run experiments and collect results
results = []
for ablation in ablations:
    for alpha in alpha_values:
        if ablation == 'Yolov8' or ablation == 'baseline':
            model_path = f'datasets/hierarchical-model-weights/weights/best_model_alpha_{alpha:.2f}.pth'
        else:
            model_path = f'datasets/hierarchical-model-weights/ablations/{ablation}/weights/best_model_alpha_{alpha:.2f}.pth'
        f1_scores = run_experiment(data_path, classes_file, model_path, ablation, root)
        results.append({'Ablation': ablation, 'Alpha': alpha, **f1_scores})

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.round(4)

,Ablation,Alpha,binary,class,genus,species
0,Yolov8,0.0,0.6608,0.8810,0.9151,0.9205
1,Yolov8,0.2,0.6608,0.8810,0.9151,0.9205
2,Yolov8,0.5,0.6608,0.8810,0.9151,0.9205
3,Yolov8,0.8,0.6608,0.8810,0.9151,0.9205
4,Yolov8,1.0,0.6608,0.8810,0.9151,0.9205
5,baseline,0.0,0.6610,0.8814,0.9162,0.9224
6,baseline,0.2,0.6613,0.8822,0.9159,0.9218
7,baseline,0.5,0.6621,0.8826,0.9167,0.9220
8,baseline,0.8,0.6615,0.8810,0.9165,0.9210
9,baseline,1.0,0.6610,0.8822,0.9164,0.9214
